In [3]:
# Import libraries and Packages
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime as dt
from datetime import datetime,tzinfo
from pytz import timezone
import time
import pytz
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pprint as pp


## Filter news by marker
This notebook is intended to perform the following processes:

    1.1 Creates marks for mapping statistically significant price changes to news articles within a window of time

___

In [4]:
marker= pd.read_csv('New_Markers.csv')
marker.head()
mark=marker['Date'] #create a new column of dates to track events
mark.head(5)

0    2018-02-05
1    2018-03-15
2    2018-03-17
3    2018-03-18
Name: Date, dtype: object

__Modify datestamp for mapping purposes__

In [5]:
# create list of timestamps according to given datetime parameters
array_mark = []
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date)
pp.pprint(array_mark[0:5])

[datetime.datetime(2018, 2, 5, 0, 0),
 datetime.datetime(2018, 3, 15, 0, 0),
 datetime.datetime(2018, 3, 17, 0, 0),
 datetime.datetime(2018, 3, 18, 0, 0)]


__Allows for a given date range of consideration (+- N days from statistically significant price change)__

In [6]:
N = 3
days_before=[]
days_after=[]
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    days_before.append((date-timedelta(days=N)).isoformat())
    days_after.append((date+timedelta(days=N)).isoformat())

In [7]:
# appends additional days to list -- used for mapping of articles
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date-timedelta(days=1))
    array_mark.append(date+timedelta(days=1))
    array_mark.append(date-timedelta(days=2))
    array_mark.append(date+timedelta(days=2))
    array_mark.append(date-timedelta(days=3))
    array_mark.append(date+timedelta(days=3))
pp.pprint(array_mark[:10])

[datetime.datetime(2018, 2, 5, 0, 0),
 datetime.datetime(2018, 3, 15, 0, 0),
 datetime.datetime(2018, 3, 17, 0, 0),
 datetime.datetime(2018, 3, 18, 0, 0),
 datetime.datetime(2018, 2, 4, 0, 0),
 datetime.datetime(2018, 2, 6, 0, 0),
 datetime.datetime(2018, 2, 3, 0, 0),
 datetime.datetime(2018, 2, 7, 0, 0),
 datetime.datetime(2018, 2, 2, 0, 0),
 datetime.datetime(2018, 2, 8, 0, 0)]


In [8]:
# 1x8 array of 1's
np.ones(8)

array([1., 1., 1., 1., 1., 1., 1., 1.])

__Mark 'significant' dates__

In [9]:
# Marking of dates with statistically significant price changes
date_mark=pd.DataFrame({'Date':array_mark,
                       'Mark':np.ones(len(array_mark))})
pp.pprint(date_mark.head())
date_mark.to_csv('positive_marked_dates.csv')

        Date  Mark
0 2018-02-05   1.0
1 2018-03-15   1.0
2 2018-03-17   1.0
3 2018-03-18   1.0
4 2018-02-04   1.0


In [10]:
## checking variable types ##
#type(date_mark['Date'][0])  # .Timestamp
#type(date_mark['Mark'][0]) # float type

__Prepare articles for marking__

In [11]:
df= pd.read_csv('article_data_and_price_labeled_publisher.csv').drop('Unnamed: 0', axis = 1)
df

,author,contents,description,publisher,source_url,timeStamp,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,959
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,365
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1181
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,241
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,452
5,Financial Times,hannah murphy london bitflyer tokyo based oper...,Exchange to offer trading of bitcoin/euro pair...,Financial Times,https://www.ft.com/content/5bf1462c-ff84-11e7-...,2018-01-23 00:06:21+00:00,Japan’s bitFlyer set to stage Europe expansion,2018-01-23,00:06:21,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,364
6,Topix.com,not topix user yet forgot your password news p...,The fundamentals are as untenable as they were...,Topix.com,http://www.topix.com/tech/p2p/2018/01/dont-try...,2018-01-23 00:11:59+00:00,Don't Try To Catch The Bitcoin Knife,2018-01-23,00:11:59,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1085
7,Chloe Aiello,var postloadfunctions var foresee enabled var ...,ETF Managers Group Founder and CEO Sam Masucci...,CNBC,https://www.cnbc.com/2018/01/22/marijuana-etf-...,2018-01-23 00:26:00+00:00,Here's why one investor is way more comfortabl...,2018-01-23,00:26:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,146
8,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,Miami has apparently witnessed the “hottest” w...,Bitcoinist.com,http://bitcoinist.com/achain-showcased-collabo...,2018-01-23 00:30:10+00:00,Achain Showcased its Collaborative Platform at...,2018-01-23,00:30:10,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,96
9,Sasha Lekach,the bitcoin craze officially jumped real estat...,The bitcoin craze has officially jumped to rea...,Mashable,http://mashable.com/2018/01/22/cryptohomes-rea...,2018-01-23 

In [12]:
#Convert Timestamp into separate Date and Time
df['Date'] = pd.to_datetime(df['timeStamp']).dt.date

del df['timeStamp'] #Delete original datetime column
df.head()
print(date_mark)

         Date  Mark
0  2018-02-05   1.0
1  2018-03-15   1.0
2  2018-03-17   1.0
3  2018-03-18   1.0
4  2018-02-04   1.0
5  2018-02-06   1.0
6  2018-02-03   1.0
7  2018-02-07   1.0
8  2018-02-02   1.0
9  2018-02-08   1.0
10 2018-03-14   1.0
11 2018-03-16   1.0
12 2018-03-13   1.0
13 2018-03-17   1.0
14 2018-03-12   1.0
15 2018-03-18   1.0
16 2018-03-16   1.0
17 2018-03-18   1.0
18 2018-03-15   1.0
19 2018-03-19   1.0
20 2018-03-14   1.0
21 2018-03-20   1.0
22 2018-03-17   1.0
23 2018-03-19   1.0
24 2018-03-16   1.0
25 2018-03-20   1.0
26 2018-03-15   1.0
27 2018-03-21   1.0


__Additional Pre-Processing__

In [13]:
# datatype coercion of features
df.date = df.date.astype(str)               
date_mark.Date = date_mark.Date.astype(str)

__Mark articles according to statistically significant price changes__

In [14]:
marked = pd.merge(df,date_mark, how='left', left_on='date',right_on='Date')

In [15]:
# handle missing marks
marked.Mark = marked.Mark.fillna(0)

In [19]:
df_1=marked[marked['Mark']==1]
df_1

,author,contents,description,publisher,source_url,title,date,time,Open,High,...,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date_x,Date_y,Mark
3270,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,With Israel’s Finance Ministry and the Bank of...,Bitcoinist.com,http://bitcoinist.com/kosher-crypto-bitcoen-se...,Kosher Crypto BitCoen Is Setting a Course for ...,2018-02-02,00:00:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,96,2018-02-02,2018-02-02,1.0
3271,Michelle Fox,var postloadfunctions var foresee enabled var ...,"Bitcoin may still drop to $7,500, but it will ...",CNBC,https://www.cnbc.com/2018/02/01/bitcoin-near-b...,"Bitcoin near bottom, will rally to $20,000 thi...",2018-02-02,00:02:00,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,146,2018-02-02,2018-02-02,1.0
3272,Scott Scanlon,at core cryptocurrency networks miners people ...,At the core of most cryptocurrency networks ar...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/brai...,Brain Genius Submerges His Bitcoin Mining Rig ...,2018-02-02,00:03:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0
3273,Bruce Kleinman,demons digital gold part if already done pleas...,"Demons in Digital Gold, Part 5",Hackernoon.com,https://hackernoon.com/remediation-wherefore-a...,"Remediation, wherefore art thou?",2018-02-02,00:18:34,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,452,2018-02-02,2018-02-02,1.0
3274,Jason Murphy,email password remember me feb while systems u...,While the systems underpinning bitcoin are tru...,Crikey.com.au,https://www.crikey.com.au/2018/02/02/cryptotra...,Cryptotragedy: what if bitcoin’s greatest stre...,2018-02-02,00:25:09,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,220,2018-02-02,2018-02-02,1.0
3275,CoinTelegraph By Molly Jane Zuckerman,cointelegraph jordan belfort wolf wall street ...,"Jordan Belfort, the “Wolf of Wall Street”, cal...",Cointelegraph.com,https://cointelegraph.com/news/wolf-of-wall-st...,Wolf Of Wall Street Says Bitcoin Could Hit $50...,2018-02-02,00:49:21,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,199,2018-02-02,2018-02-02,1.0
3276,Scott Scanlon,timothy b lee pm bitcoin month long price slid...,"Timothy B. Lee – 2/1/2018, 8:15 PM Bitcoin’s m...",Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/more...,More bad news pushes bitcoin’s value below $9000,2018-02-02,01:03:11,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0
3277,King5.com,seattle weather summary degrees bitcoin droppe...,"Bitcoin drops below key $9,000 level on Coinbase",King5.com,http://www.king5.com/article/news/nation-now/b...,"Bitcoin drops below key $9,000 level on Coinbase",2018-02-02,01:40:17,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,584,2018-02-02,2018-02-02,1.0
3278,Jonathan Berr,bitcoin dropped friday reflecting plunge almos...,Bitcoin has lost almost 60 percent of its valu...,CBS News,https://www.cbsnews.com/news/cryptocurrency-pr...,Cryptocurrency prices plunge as regulators cla...,2018-02-02,01:42:56,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,145,2018-02-02,2018-02-02,1.0
3279,Duncan Riley,home news duncan riley updated est february th...,The price of bitcoin plunged in trading Thursd...,Siliconangle.com,https://siliconangle.com/blog/2018/02/01/bitco...,Bitcoin price plunges as India announces ban o...,2018-02-02,02:02:58,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647

In [20]:
df_1.to_csv('New_marked_news.csv')

___